In [4]:
import os
import sys
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [5]:
import util

In [6]:
import tensorflow as tf
tf.__version__

'1.13.1'

In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Fri_Feb__8_19:08:26_Pacific_Standard_Time_2019
Cuda compilation tools, release 10.1, V10.1.105


In [8]:
from keras_retinanet import models

In [9]:
with open('settings.json') as json_data_file:
    json_data = json.load(json_data_file)

In [10]:
model1_path = json_data["MODEL_50"]
model1 = models.load_model(model1_path, backbone_name='resnet50', convert=True, nms=False)

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
model2_path = json_data["MODEL_101"]
model2 = models.load_model(model2_path, backbone_name='resnet101', convert=True, nms=False)

In [12]:
sz = 224

# threshold for non-max-suppresion for each model
nms_threshold = 0

# shrink bounding box dimensions by this factor, improves test set performance
shrink_factor = 0.17

# threshold for judging overlap of bounding boxes between different networks (for weighted average)
wt_overlap = 0

# threshold for including boxes from model 1
score_threshold1 = 0.04

# threshold for including boxes from model 2
score_threshold2 = 0.05

# threshold for including isolated boxes from either model
solo_min = 0.15

In [13]:
import time
import cv2
import os

In [14]:
file_path=[]
score_list=[]
d = "data/DR/"
for path in os.listdir(d):
    full_path = os.path.join(d, path)
    if os.path.isfile(full_path):
        file_path.append(path)
        #fid = fname[:-4]
        start = time.time()
        fpath = d+path

        boxes_pred1, scores1 = util.get_detection_from_file(fpath, model1, sz)
        boxes_pred2, scores2 = util.get_detection_from_file(fpath, model2, sz)


        indices1 = np.where(scores1 > score_threshold1)[0]
        scores1 = scores1[indices1]
        boxes_pred1 = boxes_pred1[indices1]
        boxes_pred1, scores1 = util.nms(boxes_pred1, scores1, nms_threshold)

        indices2 = np.where(scores2 > score_threshold2)[0]
        scores2 = scores2[indices2]
        boxes_pred2 = boxes_pred2[indices2]
        boxes_pred2, scores2 = util.nms(boxes_pred2, scores2, nms_threshold)

        #boxes_pred = boxes_pred1+boxes_pred2
        #scores = scores1+scores2
        
        boxes_pred = np.concatenate((boxes_pred1, boxes_pred2))
        scores = np.concatenate((scores1, scores2))

        boxes_pred, scores = util.averages(
        boxes_pred, scores, wt_overlap, solo_min)
        util.shrink(boxes_pred, shrink_factor)
        
        score_list.append(scores)
        
        boxx = []
        #print(boxes_pred,scores)

        for i, bb in enumerate(boxes_pred):
            x1 = int(bb[0])
            y1 = int(bb[1])
            w = int(bb[2]-x1+1)
            h = int(bb[3]-y1+1)

            boxx.append([x1,y1,w,h])

        im = cv2.imread(fpath)
        util.draw(im,boxx)

In [15]:
print(len(score_list))

140


In [16]:
data = {'col_1': file_path, 'col_2': score_list}
df=pd.DataFrame.from_dict(data)

In [58]:
df.to_csv('out.csv')

In [17]:
#fid = fname[:-4]
start = time.time()
fpath = './test/05.jpg'

#boxes_pred1, scores1 = util.get_detection_from_file(fpath, model1, sz)
boxes_pred2, scores2 = util.get_detection_from_file(fpath, model2, sz)


# indices1 = np.where(scores1 > score_threshold1)[0]
# scores1 = scores1[indices1]
# boxes_pred1 = boxes_pred1[indices1]
# boxes_pred1, scores1 = util.nms(boxes_pred1, scores1, nms_threshold)

indices2 = np.where(scores2 > score_threshold2)[0]
scores2 = scores2[indices2]
boxes_pred2 = boxes_pred2[indices2]
boxes_pred2, scores2 = util.nms(boxes_pred2, scores2, nms_threshold)

boxes_pred = boxes_pred2
scores = scores2

boxx = []
#print(boxes_pred,scores)

for i, bb in enumerate(boxes_pred):
    x1 = int(bb[0])
    y1 = int(bb[1])
    w = int(bb[2]-x1+1)
    h = int(bb[3]-y1+1)
    
    boxx.append([x1,y1,w,h])

In [18]:
import cv2
im = cv2.imread(fpath)

In [19]:
print(scores)

[0.5625771]


In [20]:
util.draw(im,boxx)

In [22]:
print(im)

[[[  2   2   2]
  [  1   1   1]
  [  1   1   1]
  ...
  [  3   3   3]
  [  3   3   3]
  [  2   2   2]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  1   1   1]
  [  0   0   0]]

 [[  2   2   2]
  [  2   2   2]
  [  3   3   3]
  ...
  [ 13  13  13]
  [  9   9   9]
  [  2   2   2]]

 ...

 [[ 30  30  30]
  [ 31  31  31]
  [ 37  37  37]
  ...
  [119 119 119]
  [ 73  73  73]
  [ 27  27  27]]

 [[ 23  23  23]
  [ 24  24  24]
  [ 28  28  28]
  ...
  [100 100 100]
  [ 64  64  64]
  [ 28  28  28]]

 [[ 16  16  16]
  [ 15  15  15]
  [ 17  17  17]
  ...
  [ 51  51  51]
  [ 31  31  31]
  [  8   8   8]]]


In [11]:
#densenet

In [25]:
from tensorflow.keras.models import load_model as tf_load_model

In [23]:
from keras.models import load_model as keras_load_model

In [28]:
model = keras_load_model('densenet121_m_f_14_14.h5')

TypeError: __init__() got an unexpected keyword argument 'ragged'

In [32]:
!pip freeze

absl-py==0.7.1
alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.6
anaconda-project==0.8.2
asn1crypto==0.24.0
astor==0.7.1
astroid==2.1.0
astropy==3.1
atomicwrites==1.2.1
attrs==18.2.0
Automat==0.7.0
Babel==2.6.0
backcall==0.1.0
backports.os==0.1.1
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.6.3
bitarray==0.8.3
bkcharts==0.2
blaze==0.11.3
bleach==3.0.2
bokeh==1.0.2
boto==2.49.0
Bottleneck==1.2.1
boxx==0.9.2.13
catboost==0.13.1
category-encoders==1.3.0
certifi==2019.6.16
cffi==1.12.3
chardet==3.0.4
Click==7.0
cloudpickle==0.6.1
clyent==1.2.2
colorama==0.4.1
comtypes==1.1.7
conda==4.7.9
conda-build==3.17.6
conda-package-handling==1.3.11
conda-verify==3.1.1
constantly==15.1.0
contextlib2==0.5.5
cryptography==2.7
cycler==0.10.0
Cython==0.29.2
cytoolz==0.9.0.1
dask==1.0.0
datashape==0.5.4
decorator==4.3.0
defusedxml==0.5.0
distributed==1.25.1
docutils==0.14
entrypoints==0.2.3
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==3.0.10
Flask==1.0.2
Flask-Cors==3.0.7
fn

# Requrements###
absl-py==0.7.1
alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.6
anaconda-project==0.8.2
asn1crypto==0.24.0
astor==0.7.1
astroid==2.1.0
astropy==3.1
atomicwrites==1.2.1
attrs==18.2.0
Automat==0.7.0
Babel==2.6.0
backcall==0.1.0
backports.os==0.1.1
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.6.3
bitarray==0.8.3
bkcharts==0.2
blaze==0.11.3
bleach==3.0.2
bokeh==1.0.2
boto==2.49.0
Bottleneck==1.2.1
boxx==0.9.2.13
catboost==0.13.1
category-encoders==1.3.0
certifi==2019.6.16
cffi==1.12.3
chardet==3.0.4
Click==7.0
cloudpickle==0.6.1
clyent==1.2.2
colorama==0.4.1
comtypes==1.1.7
conda==4.7.9
conda-build==3.17.6
conda-package-handling==1.3.11
conda-verify==3.1.1
constantly==15.1.0
contextlib2==0.5.5
cryptography==2.7
cycler==0.10.0
Cython==0.29.2
cytoolz==0.9.0.1
dask==1.0.0
datashape==0.5.4
decorator==4.3.0
defusedxml==0.5.0
distributed==1.25.1
docutils==0.14
entrypoints==0.2.3
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==3.0.10
Flask==1.0.2
Flask-Cors==3.0.7
fn==0.4.3
future==0.17.1
gast==0.2.2
gevent==1.3.7
glob2==0.6
greenlet==0.4.15
grpcio==1.12.1
h5py==2.8.0
heapdict==1.0.0
html5lib==1.0.1
hyperlink==19.0.0
idna==2.8
imageio==2.4.1
imagesize==1.1.0
importlib-metadata==0.6
incremental==17.5.0
ipykernel==5.1.0
ipython==7.2.0
ipython-genutils==0.2.0
ipywidgets==7.4.2
isort==4.3.4
itsdangerous==1.1.0
jdcal==1.4
jedi==0.13.2
Jinja2==2.10
jsonschema==2.6.0
jupyter==1.0.0
jupyter-client==5.2.4
jupyter-console==6.0.0
jupyter-core==4.4.0
jupyterlab==0.35.3
jupyterlab-server==0.2.0
Keras==2.2.4
Keras-Applications==1.0.7
Keras-Preprocessing==1.0.9
keras-resnet==0.1.0
keras-retinanet==0.4.1
keyring==17.0.0
kiwisolver==1.0.1
lazy-object-proxy==1.3.1
libarchive-c==2.8
lightgbm==2.2.2
llvmlite==0.26.0
locket==0.2.0
lxml==4.2.5
Markdown==3.1
MarkupSafe==1.1.0
matplotlib==3.0.2
mccabe==0.6.1
menuinst==1.4.16
mistune==0.8.4
mkl-fft==1.0.6
mkl-random==1.0.2
mock==2.0.0
more-itertools==4.3.0
mpmath==1.1.0
msgpack==0.5.6
multipledispatch==0.6.0
navigator-updater==0.2.1
nbconvert==5.4.0
nbformat==4.4.0
networkx==2.2
nltk==3.4
nose==1.3.7
notebook==5.7.4
numba==0.41.0
numexpr==2.6.8
numpy==1.15.4
numpydoc==0.8.0
odo==0.5.1
olefile==0.46
opencv-python==4.1.0.25
openpyxl==2.5.12
packaging==18.0
pandas==0.23.4
pandocfilters==1.4.2
parso==0.3.1
partd==0.3.9
path.py==11.5.0
pathlib2==2.3.3
patsy==0.5.1
pbr==5.1.3
pep8==1.7.1
pickleshare==0.7.5
Pillow==5.3.0
pkginfo==1.4.2
plotly==3.5.0
pluggy==0.8.0
ply==3.11
pprofile==2.0.2
progressbar2==3.42.0
prometheus-client==0.5.0
prompt-toolkit==2.0.7
protobuf==3.7.1
psutil==5.4.8
py==1.7.0
py-heat==0.0.6
pycodestyle==2.4.0
pycosat==0.6.3
pycountry==18.12.8
pycparser==2.19
pycrypto==2.6.1
pycurl==7.43.0.2
pydicom==1.2.2
pydotplus==2.0.2
pyflakes==2.0.0
Pygments==2.3.1
PyHamcrest==1.9.0
pylint==2.2.2
pyodbc==4.0.25
PyOpenGL==3.1.0
pyOpenSSL==19.0.0
pyparsing==2.3.0
PySocks==1.7.0
pytest==4.0.2
pytest-arraydiff==0.3
pytest-astropy==0.5.0
pytest-doctestplus==0.2.0
pytest-openfiles==0.3.1
pytest-remotedata==0.3.1
python-dateutil==2.7.5
python-utils==2.3.0
pytz==2018.7
PyWavelets==1.0.1
pywin32==223
pywinpty==0.5.5
PyYAML==3.13
pyzmq==17.1.2
QtAwesome==0.5.3
qtconsole==4.4.3
QtPy==1.5.2
requests==2.22.0
retrying==1.3.3
rope==0.11.0
ruamel-yaml==0.15.46
scikit-image==0.14.1
scikit-learn==0.20.1
scipy==1.1.0
seaborn==0.9.0
Send2Trash==1.5.0
simplegeneric==0.8.1
singledispatch==3.4.0.3
six==1.12.0
snakeviz==2.0.1
snowballstemmer==1.2.1
sortedcollections==1.0.1
sortedcontainers==2.1.0
Sphinx==1.8.2
sphinxcontrib-websupport==1.1.0
spyder==3.3.2
spyder-kernels==0.3.0
SQLAlchemy==1.2.15
statsmodels==0.9.0
sympy==1.3
tables==3.4.4
tblib==1.3.2
tensorboard==1.13.1
tensorflow==1.13.1
tensorflow-estimator==1.13.0
termcolor==1.1.0
terminado==0.8.1
testpath==0.4.2
toolz==0.9.0
torch==1.0.1
torchvision==0.2.2
tornado==5.1.1
tqdm==4.32.1
traitlets==4.3.2
Twisted==19.2.0
unicodecsv==0.14.1
urllib3==1.24.2
wcwidth==0.1.7
webencodings==0.5.1
Werkzeug==0.14.1
widgetsnbextension==3.4.2
win-inet-pton==1.1.0
win-unicode-console==0.5
wincertstore==0.2
wrapt==1.10.11
xlrd==1.2.0
XlsxWriter==1.1.2
xlwings==0.15.1
xlwt==1.3.0
zict==0.1.3
zope.interface==4.6.0
​